# Time Series Aggregation (2022-2025)

This notebook creates monthly aggregated time series data by combining:
1. Historic data (2022-2024) from notebook 07
2. June 2025 data from notebook 04

## Objectives
1. Load consolidated historic data (36 months)
2. Load June 2025 processed data
3. Create monthly aggregations by Betriebszentralen
4. Merge into complete time series (42 months total)
5. Prepare data for forecasting models

## Output
- **Time range**: January 2022 - June 2025 (42 months)
- **Granularity**: Monthly aggregations by Betriebszentralen (14 dispatch centers)
- **Metrics**: Orders, KM, carriers, costs, efficiency

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Section 1: Load Historic Data (2022-2024)

Load the consolidated historic dataset created in notebook 07.

In [2]:
# Load historic data (use Parquet for faster loading)
historic_path = Path('../data/processed/historic_orders_2022_2024.parquet')

if historic_path.exists():
    print(f"Loading historic data from {historic_path}...")
    df_historic = pd.read_parquet(historic_path)
    print(f"✓ Loaded {len(df_historic):,} records")
else:
    # Try CSV.gz if Parquet doesn't exist
    historic_path_csv = Path('../data/processed/historic_orders_2022_2024.csv.gz')
    print(f"Loading historic data from {historic_path_csv}...")
    df_historic = pd.read_csv(historic_path_csv, compression='gzip')
    print(f"✓ Loaded {len(df_historic):,} records")

print(f"\nDataset info:")
print(f"  Shape: {df_historic.shape}")
print(f"  Date range: {df_historic['Datum.Tour'].min()} to {df_historic['Datum.Tour'].max()}")
print(f"  Memory: {df_historic.memory_usage(deep=True).sum() / 1024**3:.2f} GB")

Loading historic data from ../data/processed/historic_orders_2022_2024.parquet...


✓ Loaded 4,935,622 records

Dataset info:
  Shape: (4935622, 119)


  Date range: 2022-01-01 00:00:00 to 2024-12-31 00:00:00


  Memory: 28.98 GB


## Section 2: Monthly Aggregation by Betriebszentralen

Aggregate historic data to monthly level grouped by the 14 Betriebszentralen (dispatch centers).

In [3]:
print("Creating monthly aggregations...")
print("="*80)

# Ensure date column is datetime
df_historic['Datum.Tour'] = pd.to_datetime(df_historic['Datum.Tour'])

# Create year-month column
df_historic['year_month'] = df_historic['Datum.Tour'].dt.to_period('M')

# Find the order ID column (use the most complete identifier)
order_id_cols = [
  'NummerKomplett.Auftrag',  # Most complete
  'Nummer.Auftrag',           # Order number
  'Nummer.Hauptauftrag',      # Main order
  'Auftragsschein-Nr.',       # Legacy name
]

order_id_col = None
for col in order_id_cols:
  if col in df_historic.columns:
      order_id_col = col
      print(f"✓ Using order ID column: {col}")
      break

if not order_id_col:
  # Fallback: use index as count
  print("⚠️  No order ID column found, using record count")
  df_historic['_order_count'] = 1
  order_id_col = '_order_count'

# Check if betriebszentrale_name exists
if 'betriebszentrale_name' not in df_historic.columns:
  print("⚠️  'betriebszentrale_name' not found, using 'Unknown'")
  df_historic['betriebszentrale_name'] = 'Unknown'

# Check if carrier_type exists
if 'carrier_type' not in df_historic.columns:
  print("⚠️  'carrier_type' not found, classifying now...")
  # Quick carrier classification
  def classify_carrier(carrier_num):
      if pd.isna(carrier_num):
          return 'unknown'
      try:
          carrier_num = float(str(carrier_num).replace('-', '').strip())
          if carrier_num <= 8889:
              return 'internal'
          elif carrier_num >= 9000:
              return 'external'
      except:
          pass
      return 'unknown'

  df_historic['carrier_type'] = df_historic['Nummer.Spedition'].apply(classify_carrier)

# Fix: Convert distance column to numeric
print("\nPreparing columns for aggregation...")

# Convert distance to numeric (handles strings and errors)
if 'Distanz_BE.Auftrag' in df_historic.columns:
  df_historic['Distanz_BE.Auftrag'] = pd.to_numeric(
      df_historic['Distanz_BE.Auftrag'],
      errors='coerce'  # Convert invalid values to NaN
  )
  print(f"  ✓ Converted Distanz_BE.Auftrag to numeric")
  print(f"    Valid distances: {df_historic['Distanz_BE.Auftrag'].notna().sum():,}")
  print(f"    Invalid/missing: {df_historic['Distanz_BE.Auftrag'].isna().sum():,}")

# Convert revenue column to numeric if exists
if '∑ Einnahmen' in df_historic.columns:
  df_historic['∑ Einnahmen'] = pd.to_numeric(
      df_historic['∑ Einnahmen'],
      errors='coerce'  # Convert invalid values to NaN
  )
  print(f"  ✓ Converted ∑ Einnahmen (revenue) to numeric")
  print(f"    Valid revenue: {df_historic['∑ Einnahmen'].notna().sum():,}")
  print(f"    Non-zero revenue: {(df_historic['∑ Einnahmen'] > 0).sum():,}")

# Group by month and Betriebszentralen
print("\nAggregating by year-month and Betriebszentralen...")

agg_dict = {
  order_id_col: 'count',  # Total orders
  'Distanz_BE.Auftrag': 'sum',  # Total KM
}

# Add revenue if column exists
if '∑ Einnahmen' in df_historic.columns:
  agg_dict['∑ Einnahmen'] = 'sum'  # Total revenue
  print("  ✓ Including revenue_total in aggregation")

monthly_agg = df_historic.groupby(['year_month',
'betriebszentrale_name']).agg(agg_dict).reset_index()

# Rename columns
if '∑ Einnahmen' in df_historic.columns:
  monthly_agg.columns = ['year_month', 'betriebszentrale', 'total_orders', 'total_km', 'revenue_total']
else:
  monthly_agg.columns = ['year_month', 'betriebszentrale', 'total_orders', 'total_km']

# Add external drivers count
external_count = df_historic[df_historic['carrier_type'] == 'external'].groupby(
  ['year_month', 'betriebszentrale_name']
).size().reset_index(name='external_drivers')

monthly_agg = monthly_agg.merge(
  external_count,
  left_on=['year_month', 'betriebszentrale'],
  right_on=['year_month', 'betriebszentrale_name'],
  how='left'
).drop(columns=['betriebszentrale_name'])

# Add internal drivers count
internal_count = df_historic[df_historic['carrier_type'] == 'internal'].groupby(
  ['year_month', 'betriebszentrale_name']
).size().reset_index(name='internal_drivers')

monthly_agg = monthly_agg.merge(
  internal_count,
  left_on=['year_month', 'betriebszentrale'],
  right_on=['year_month', 'betriebszentrale_name'],
  how='left'
).drop(columns=['betriebszentrale_name'])

# Fill NaN with 0 for driver counts
monthly_agg['external_drivers'] = monthly_agg['external_drivers'].fillna(0).astype(int)
monthly_agg['internal_drivers'] = monthly_agg['internal_drivers'].fillna(0).astype(int)

# Add order type breakdown if available
if 'order_type' in df_historic.columns:
  order_types = df_historic.groupby(['year_month', 'betriebszentrale_name',
'order_type']).size().unstack(fill_value=0)
  order_types = order_types.reset_index()
  order_types.columns.name = None
  order_types = order_types.rename(columns={'betriebszentrale_name': 'betriebszentrale'})

  monthly_agg = monthly_agg.merge(order_types, on=['year_month', 'betriebszentrale'],
how='left')

# Convert year_month to datetime for easier plotting
monthly_agg['date'] = monthly_agg['year_month'].dt.to_timestamp()

print(f"\n✓ Monthly aggregation complete")
print(f"  Total month-entity combinations: {len(monthly_agg):,}")
print(f"  Unique months: {monthly_agg['year_month'].nunique()}")
print(f"  Unique Betriebszentralen: {monthly_agg['betriebszentrale'].nunique()}")
print(f"\n  Date range: {monthly_agg['date'].min()} to {monthly_agg['date'].max()}")

# Preview
print("\nSample of aggregated data:")
print(monthly_agg.head(10))

Creating monthly aggregations...


✓ Using order ID column: NummerKomplett.Auftrag

Preparing columns for aggregation...


  ✓ Converted Distanz_BE.Auftrag to numeric


    Valid distances: 4,847,359


    Invalid/missing: 88,263


  ✓ Converted ∑ Einnahmen (revenue) to numeric


    Valid revenue: 4,935,622


    Non-zero revenue: 3,286,951

Aggregating by year-month and Betriebszentralen...
  ✓ Including revenue_total in aggregation



✓ Monthly aggregation complete
  Total month-entity combinations: 375
  Unique months: 36
  Unique Betriebszentralen: 12

  Date range: 2022-01-01 00:00:00 to 2024-12-01 00:00:00

Sample of aggregated data:
  year_month      betriebszentrale  total_orders   total_km  revenue_total  \
0    2022-01         B&T Landquart           287     7784.0   4.897960e+04   
1    2022-01           B&T Puidoux          2455   107169.0   4.350852e+05   
2    2022-01        B&T Winterthur          4938   157079.0   7.026364e+05   
3    2022-01    BZ Herzogenbuchsee          7312   494260.0   3.119368e+06   
4    2022-01  BZ Intermodal / Rail           174    12041.0   5.796830e+04   
5    2022-01          BZ Landquart         17739  1081152.0   6.850266e+05   
6    2022-01           BZ Oberbipp         34737  2411456.0   7.960313e+05   
7    2022-01            BZ Puidoux            13     1699.0   6.403000e+03   
8    2022-01             BZ Sursee         28480  2200464.0   4.319064e+06   
9    2022-01

## Section 3: Load June 2025 Data

Load the already-processed June 2025 data from notebook 04.

In [4]:
# Check if we have processed June 2025 aggregated data
june_2025_path = Path('../data/processed/monthly_aggregated.csv')

if june_2025_path.exists():
    print(f"Loading June 2025 aggregated data from {june_2025_path}...")
    df_june_2025 = pd.read_csv(june_2025_path)
    
    # Filter for June 2025 only
    if 'year_month' in df_june_2025.columns:
        df_june_2025['year_month'] = pd.to_datetime(df_june_2025['year_month']).dt.to_period('M')
        df_june_2025 = df_june_2025[df_june_2025['year_month'] == '2025-06']
    
    print(f"✓ Loaded June 2025: {len(df_june_2025)} Betriebszentralen")
    print(f"  Columns: {list(df_june_2025.columns)}")
else:
    print("⚠️  June 2025 aggregated data not found.")
    print("    You may need to run notebook 04 first to create monthly_aggregated.csv")
    df_june_2025 = None

Loading June 2025 aggregated data from ../data/processed/monthly_aggregated.csv...


✓ Loaded June 2025: 12 Betriebszentralen
  Columns: ['year_month', 'betriebszentrale_name', 'total_orders', 'revenue_total', 'total_costs', 'external_carrier_costs', 'total_distance_km', 'avg_distance_km', 'median_distance_km', 'external_driver_orders', 'internal_driver_orders', 'delivery_orders', 'pickup_orders', 'leergut_orders', 'retoure_orders', 'unique_sparten_count', 'profit_margin', 'revenue_per_order', 'cost_per_order', 'date', 'year', 'month', 'quarter', 'month_name', 'split']


## Section 4: Merge Historic and June 2025 Data

Combine into single time series dataset.

In [5]:
# Section 4: Finalize Dataset (Historic Only)
print("Finalizing dataset (2022-2024 only)...")
print("="*80)

# Use the monthly_agg from Section 2
df_full_ts = monthly_agg.copy()

print(f"\n✓ Dataset finalized!")
print(f"  Total records: {len(df_full_ts):,}")
print(f"  Date range: {df_full_ts['date'].min()} to {df_full_ts['date'].max()}")
print(f"  Total months: {df_full_ts['year_month'].nunique()}")
print(f"  Betriebszentralen: {df_full_ts['betriebszentrale'].nunique()}")

print(f"\nAvailable metrics:")
metric_cols = [col for col in df_full_ts.columns
             if col not in ['year_month', 'date', 'betriebszentrale']]
for col in metric_cols:
  non_null = df_full_ts[col].notna().sum()
  print(f"  • {col}: {non_null:,} non-null values")

Finalizing dataset (2022-2024 only)...

✓ Dataset finalized!
  Total records: 375
  Date range: 2022-01-01 00:00:00 to 2024-12-01 00:00:00
  Total months: 36
  Betriebszentralen: 12

Available metrics:
  • total_orders: 375 non-null values
  • total_km: 375 non-null values
  • revenue_total: 375 non-null values
  • external_drivers: 375 non-null values
  • internal_drivers: 375 non-null values
  • Delivery: 375 non-null values
  • Leergut: 375 non-null values
  • Pickup/Multi-leg: 375 non-null values
  • Retoure/Abholung: 375 non-null values


In [6]:
# Check what columns we have in the merged dataset
print("\nColumns in merged dataset:")
print(df_full_ts.columns.tolist())

print(f"\nSample of merged data:")
print(df_full_ts.head(10))

print(f"\nData types:")
print(df_full_ts.dtypes)


Columns in merged dataset:
['year_month', 'betriebszentrale', 'total_orders', 'total_km', 'revenue_total', 'external_drivers', 'internal_drivers', 'Delivery', 'Leergut', 'Pickup/Multi-leg', 'Retoure/Abholung', 'date']

Sample of merged data:
  year_month      betriebszentrale  total_orders   total_km  revenue_total  \
0    2022-01         B&T Landquart           287     7784.0   4.897960e+04   
1    2022-01           B&T Puidoux          2455   107169.0   4.350852e+05   
2    2022-01        B&T Winterthur          4938   157079.0   7.026364e+05   
3    2022-01    BZ Herzogenbuchsee          7312   494260.0   3.119368e+06   
4    2022-01  BZ Intermodal / Rail           174    12041.0   5.796830e+04   
5    2022-01          BZ Landquart         17739  1081152.0   6.850266e+05   
6    2022-01           BZ Oberbipp         34737  2411456.0   7.960313e+05   
7    2022-01            BZ Puidoux            13     1699.0   6.403000e+03   
8    2022-01             BZ Sursee         28480  22004

## Section 5: Data Completeness Check

Verify we have complete time series for each Betriebszentralen.

In [7]:
print("Time Series Completeness Check")
print("="*80)

# Expected months
all_months = pd.period_range(
    start=df_full_ts['year_month'].min(), 
    end=df_full_ts['year_month'].max(), 
    freq='M'
)
print(f"\nExpected months: {len(all_months)}")
print(f"  From: {all_months[0]}")
print(f"  To: {all_months[-1]}")

# Check completeness for each Betriebszentralen
print(f"\nCompleteness by Betriebszentralen:")
print(f"{'Betriebszentralen':<30} {'Months':>8} {'Complete?':>12}")
print("-" * 52)

for bz in sorted(df_full_ts['betriebszentrale'].unique()):
    bz_data = df_full_ts[df_full_ts['betriebszentrale'] == bz]
    month_count = bz_data['year_month'].nunique()
    is_complete = "✓" if month_count == len(all_months) else "⚠️  Missing"
    print(f"{bz:<30} {month_count:>8} {is_complete:>12}")

# Check for missing months
missing_data = []
for bz in df_full_ts['betriebszentrale'].unique():
    bz_months = set(df_full_ts[df_full_ts['betriebszentrale'] == bz]['year_month'])
    missing_months = set(all_months) - bz_months
    if missing_months:
        missing_data.append((bz, sorted(missing_months)))

if missing_data:
    print(f"\n⚠️  Missing Data Details:")
    for bz, months in missing_data:
        print(f"  {bz}: {len(months)} missing months")
        if len(months) <= 5:
            print(f"    → {', '.join(str(m) for m in months)}")
else:
    print(f"\n✓ All Betriebszentralen have complete time series!")

Time Series Completeness Check

Expected months: 36
  From: 2022-01
  To: 2024-12

Completeness by Betriebszentralen:
Betriebszentralen                Months    Complete?
----------------------------------------------------
B&T Landquart                        36            ✓
B&T Puidoux                          36            ✓
B&T Winterthur                       36            ✓
BZ Herzogenbuchsee                   36            ✓
BZ Intermodal / Rail                 36            ✓
BZ Landquart                         36            ✓
BZ Oberbipp                          36            ✓
BZ Puidoux                           36            ✓
BZ Sierre                             9  ⚠️  Missing
BZ Sursee                            36            ✓
BZ Winterthur                        36            ✓
Unknown                               6  ⚠️  Missing

⚠️  Missing Data Details:
  Unknown: 30 missing months
  BZ Sierre: 27 missing months


In [8]:
# Filter: Remove "Unknown" Betriebszentralen
print("\n" + "="*80)
print("FILTERING: Remove Unknown Betriebszentralen")
print("="*80)

# Count before
before_count = len(df_full_ts)
unknown_count = len(df_full_ts[df_full_ts['betriebszentrale'] == 'Unknown'])

# Filter out Unknown
df_full_ts = df_full_ts[df_full_ts['betriebszentrale'] != 'Unknown'].copy()

# Count after
after_count = len(df_full_ts)

print(f"\n✓ Filtering complete:")
print(f"  Before: {before_count:,} records")
print(f"  Removed (Unknown): {unknown_count:,} records")
print(f"  After: {after_count:,} records")
print(f"  Retention: {(after_count/before_count)*100:.1f}%")

print(f"\nFinal entities ({df_full_ts['betriebszentrale'].nunique()}):")
for entity in sorted(df_full_ts['betriebszentrale'].unique()):
  month_count = df_full_ts[df_full_ts['betriebszentrale'] == entity]['year_month'].nunique()
  print(f"  • {entity}: {month_count} months")


FILTERING: Remove Unknown Betriebszentralen

✓ Filtering complete:
  Before: 375 records
  Removed (Unknown): 6 records
  After: 369 records
  Retention: 98.4%

Final entities (11):
  • B&T Landquart: 36 months
  • B&T Puidoux: 36 months
  • B&T Winterthur: 36 months
  • BZ Herzogenbuchsee: 36 months
  • BZ Intermodal / Rail: 36 months
  • BZ Landquart: 36 months
  • BZ Oberbipp: 36 months
  • BZ Puidoux: 36 months
  • BZ Sierre: 9 months
  • BZ Sursee: 36 months
  • BZ Winterthur: 36 months


## Section 6: Aggregate to Company Level

Create company-wide time series (sum across all Betriebszentralen).

In [9]:
print("Creating company-level aggregation...")
print("="*80)

# Define aggregation dict
agg_cols = {
    'total_orders': 'sum',
    'total_km': 'sum',
    'external_drivers': 'sum',
    'internal_drivers': 'sum'
}

# Add revenue if it exists in the dataframe
if 'revenue_total' in df_full_ts.columns:
    agg_cols['revenue_total'] = 'sum'
    print("  ✓ Including revenue_total in company aggregation")

# Aggregate across all Betriebszentralen
company_ts = df_full_ts.groupby('year_month').agg(agg_cols).reset_index()

# Add order types if available
if 'Delivery' in df_full_ts.columns:
    order_type_cols = [col for col in df_full_ts.columns if col in ['Delivery', 'Pickup/Multi-leg', 'Leergut', 'Retoure/Abholung']]
    order_type_sum = df_full_ts.groupby('year_month')[order_type_cols].sum().reset_index()
    company_ts = company_ts.merge(order_type_sum, on='year_month', how='left')

# Convert to datetime
company_ts['date'] = company_ts['year_month'].dt.to_timestamp()

# Calculate total drivers
company_ts['total_drivers'] = company_ts['external_drivers'] + company_ts['internal_drivers']

# Calculate KM per order
company_ts['km_per_order'] = company_ts['total_km'] / company_ts['total_orders']

# Calculate revenue per order if revenue exists
if 'revenue_total' in company_ts.columns:
    company_ts['revenue_per_order'] = company_ts['revenue_total'] / company_ts['total_orders']
    print(f"  ✓ Calculated revenue_per_order metric")

print(f"\n✓ Company-level time series created")
print(f"  Months: {len(company_ts)}")
print(f"  Date range: {company_ts['date'].min()} to {company_ts['date'].max()}")

# Show metrics including revenue if available
metrics_to_show = ['total_orders', 'total_km', 'total_drivers']
if 'revenue_total' in company_ts.columns:
    metrics_to_show.append('revenue_total')
    
print(f"\nSummary statistics:")
print(company_ts[metrics_to_show].describe())

Creating company-level aggregation...
  ✓ Including revenue_total in company aggregation
  ✓ Calculated revenue_per_order metric

✓ Company-level time series created
  Months: 36
  Date range: 2022-01-01 00:00:00 to 2024-12-01 00:00:00

Summary statistics:
        total_orders      total_km  total_drivers  revenue_total
count      36.000000  3.600000e+01      36.000000   3.600000e+01
mean   137100.444444  8.629973e+06  134685.666667   1.290871e+07
std      6429.356437  3.991910e+05    6214.328618   8.684005e+05
min    126035.000000  7.975456e+06  123611.000000   1.151280e+07
25%    132394.500000  8.343926e+06  130207.000000   1.223254e+07
50%    136173.500000  8.577382e+06  133873.000000   1.282729e+07
75%    140024.750000  8.834657e+06  137501.500000   1.346180e+07
max    154641.000000  9.729826e+06  150972.000000   1.510441e+07


## Section 7: Time Series Visualization

Visualize key metrics over time.

In [10]:
# Create multi-panel time series dashboard
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        'Total Orders Over Time',
        'Total Kilometers Traveled',
        'External vs Internal Drivers'
    ),
    vertical_spacing=0.12
)

# 1. Total Orders
fig.add_trace(
    go.Scatter(
        x=company_ts['date'],
        y=company_ts['total_orders'],
        mode='lines+markers',
        name='Total Orders',
        line=dict(color='#1f77b4', width=2)
    ),
    row=1, col=1
)

# 2. Total KM
fig.add_trace(
    go.Scatter(
        x=company_ts['date'],
        y=company_ts['total_km'],
        mode='lines+markers',
        name='Total KM',
        line=dict(color='#ff7f0e', width=2)
    ),
    row=2, col=1
)

# 3. Drivers (stacked)
fig.add_trace(
    go.Scatter(
        x=company_ts['date'],
        y=company_ts['internal_drivers'],
        mode='lines',
        name='Internal Drivers',
        line=dict(color='#2ca02c', width=0),
        fill='tozeroy'
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=company_ts['date'],
        y=company_ts['external_drivers'],
        mode='lines',
        name='External Drivers',
        line=dict(color='#d62728', width=0),
        fill='tonexty'
    ),
    row=3, col=1
)

# Update layout
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Orders", row=1, col=1)
fig.update_yaxes(title_text="Kilometers", row=2, col=1)
fig.update_yaxes(title_text="Driver Count", row=3, col=1)

fig.update_layout(
    height=900,
    title_text="Traveco Transport Metrics - Complete Time Series (2022-2025)",
    showlegend=True,
    hovermode='x unified'
)

fig.show()

# Save to results
results_dir = Path('../results')
results_dir.mkdir(exist_ok=True)
fig.write_html(results_dir / 'time_series_overview.html')
print(f"\n✓ Saved interactive chart to results/time_series_overview.html")


✓ Saved interactive chart to results/time_series_overview.html


## Section 8: Seasonality Analysis

Identify seasonal patterns in the data.

In [11]:
# Extract month from date
company_ts['month'] = company_ts['date'].dt.month
company_ts['year'] = company_ts['date'].dt.year

# Calculate monthly averages
monthly_avg = company_ts.groupby('month').agg({
    'total_orders': 'mean',
    'total_km': 'mean',
    'total_drivers': 'mean'
}).reset_index()

# Month names
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_avg['month_name'] = monthly_avg['month'].apply(lambda x: month_names[x-1])

# Create seasonality chart
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=monthly_avg['month_name'],
        y=monthly_avg['total_orders'],
        name='Avg Orders',
        marker_color='#1f77b4'
    )
)

fig.update_layout(
    title="Seasonal Pattern - Average Orders by Month (2022-2025)",
    xaxis_title="Month",
    yaxis_title="Average Monthly Orders",
    height=500
)

fig.show()

# Identify peak and low months
peak_month = monthly_avg.loc[monthly_avg['total_orders'].idxmax(), 'month_name']
low_month = monthly_avg.loc[monthly_avg['total_orders'].idxmin(), 'month_name']

print(f"\nSeasonality Insights:")
print(f"  Peak month: {peak_month} ({monthly_avg['total_orders'].max():.0f} orders)")
print(f"  Low month: {low_month} ({monthly_avg['total_orders'].min():.0f} orders)")
print(f"  Seasonal variation: {(monthly_avg['total_orders'].max() / monthly_avg['total_orders'].min() - 1) * 100:.1f}%")


Seasonality Insights:
  Peak month: Mar (147598 orders)
  Low month: Feb (130524 orders)
  Seasonal variation: 13.1%


## Section 9: Save Final Datasets

Save both Betriebszentralen-level and company-level time series.

In [12]:
output_dir = Path('../data/processed')

# 1. Save Betriebszentralen-level time series
bz_output = output_dir / 'monthly_aggregated_full_bz.csv'
print(f"Saving Betriebszentralen-level time series to {bz_output}...")
df_full_ts.to_csv(bz_output, index=False)
print(f"✓ Saved {len(df_full_ts):,} records")

# 2. Save company-level time series
company_output = output_dir / 'monthly_aggregated_full_company.csv'
print(f"\nSaving company-level time series to {company_output}...")
company_ts.to_csv(company_output, index=False)
print(f"✓ Saved {len(company_ts)} months")

# 3. Save as Parquet for faster loading in forecasting notebooks
company_ts.to_parquet(output_dir / 'monthly_aggregated_full_company.parquet', index=False)
print(f"✓ Saved Parquet version")

print(f"\n{'='*80}")
print(f"TIME SERIES AGGREGATION COMPLETE!")
print(f"{'='*80}")
print(f"\nDatasets ready for forecasting:")
print(f"  1. Betriebszentralen-level: {len(df_full_ts):,} month-entity records")
print(f"  2. Company-level: {len(company_ts)} monthly records")
print(f"  3. Time range: {company_ts['date'].min()} to {company_ts['date'].max()}")
print(f"  4. Total months: {len(company_ts)}")
print(f"\nNext step: Proceed to forecasting notebooks (09-15)")

Saving Betriebszentralen-level time series to ../data/processed/monthly_aggregated_full_bz.csv...


✓ Saved 369 records

Saving company-level time series to ../data/processed/monthly_aggregated_full_company.csv...


✓ Saved 36 months


✓ Saved Parquet version

TIME SERIES AGGREGATION COMPLETE!

Datasets ready for forecasting:
  1. Betriebszentralen-level: 369 month-entity records
  2. Company-level: 36 monthly records
  3. Time range: 2022-01-01 00:00:00 to 2024-12-01 00:00:00
  4. Total months: 36

Next step: Proceed to forecasting notebooks (09-15)


## Section 10: Summary Statistics

Final overview of the complete time series.

In [13]:
print("Complete Time Series Summary")
print("="*80)

print(f"\n1. Coverage:")
print(f"   Start date: {company_ts['date'].min()}")
print(f"   End date: {company_ts['date'].max()}")
print(f"   Total months: {len(company_ts)}")
print(f"   Years covered: {sorted(company_ts['year'].unique())}")

print(f"\n2. Totals (across all months):")
print(f"   Total orders: {company_ts['total_orders'].sum():,.0f}")
print(f"   Total KM: {company_ts['total_km'].sum():,.0f}")
print(f"   Avg KM/order: {(company_ts['total_km'].sum() / company_ts['total_orders'].sum()):.1f}")

print(f"\n3. Monthly Averages:")
print(f"   Avg orders/month: {company_ts['total_orders'].mean():,.0f}")
print(f"   Avg KM/month: {company_ts['total_km'].mean():,.0f}")
print(f"   Avg drivers/month: {company_ts['total_drivers'].mean():.0f}")

print(f"\n4. Betriebszentralen:")
print(f"   Total entities: {df_full_ts['betriebszentrale'].nunique()}")
print(f"   List: {sorted(df_full_ts['betriebszentrale'].unique())}")

print(f"\n5. Data Quality:")
print(f"   Missing values: {company_ts.isna().sum().sum()}")
print(f"   Duplicate months: {company_ts['year_month'].duplicated().sum()}")

print(f"\n✓ Dataset is ready for time series forecasting!")

Complete Time Series Summary

1. Coverage:
   Start date: 2022-01-01 00:00:00
   End date: 2024-12-01 00:00:00
   Total months: 36
   Years covered: [np.int32(2022), np.int32(2023), np.int32(2024)]

2. Totals (across all months):
   Total orders: 4,935,616
   Total KM: 310,679,019
   Avg KM/order: 62.9

3. Monthly Averages:
   Avg orders/month: 137,100
   Avg KM/month: 8,629,973
   Avg drivers/month: 134686

4. Betriebszentralen:
   Total entities: 11
   List: ['B&T Landquart', 'B&T Puidoux', 'B&T Winterthur', 'BZ Herzogenbuchsee', 'BZ Intermodal / Rail', 'BZ Landquart', 'BZ Oberbipp', 'BZ Puidoux', 'BZ Sierre', 'BZ Sursee', 'BZ Winterthur']

5. Data Quality:
   Missing values: 0
   Duplicate months: 0

✓ Dataset is ready for time series forecasting!
